In [1]:
import torch
from model import SimpleModel, ModelCT
from torch.utils.data import DataLoader
from datareader import DataReader
from sklearn.metrics import roc_auc_score
import numpy as np
from utils import read_from_pickle

In [30]:
main_path_to_data = "processed"

test_info = read_from_pickle(main_path_to_data + "/test_info")

In [31]:
len(test_info)

900

In [32]:
Model = SimpleModel()
Model.load_state_dict(torch.load("LAST_model.pth"))
Model.eval() # zaklenemo dropout plasti in normalizacijo serij

SimpleModel(
  (conv1): Conv2d(1, 4, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=65536, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=1, bias=True)
)

In [16]:
# DataLoader za testno mnozico:
test_datareader = DataReader(main_path_to_data, test_info)
test_generator = DataLoader(test_datareader, batch_size=10, shuffle=False, pin_memory=True, num_workers=4)

TypeError: object of type 'generator' has no len()

In [24]:
predictions = [] # seznam nasih napovedi
trues = [] # seznam dejanskih oznak

for item_test in test_generator:
    # Verjetnost za dani batch
    prediction = Model.predict(item_test, is_prob=True)
    # Izracunamo povprecje po vseh 10ih rezinah v batchu
    predictions.append(np.mean(prediction.cpu().numpy()))
    #print("Truth:", item_test[1].numpy()[0])
    trues.append(item_test[1].numpy()[0])

Truth: [1.]
Truth: [0.]


In [26]:
# Izracunamo AUC
#auc = roc_auc_score(trues, predictions)